In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Task 1: K-Means
Implement k-Means from scratch with the euclidean distance.
Complete the functions recompute_centroids, recompute_assigments and k_means.
Implement early stopping when the algorithm has converged to a solution.
Use the numpy arrays in the code to represent the cluster assigments and centroids.

In [ ]:
def k_means(X, k, centroids,n_iter=100):
    labels = np.zeros(X.shape[0])
    for i in range(n_iter):
        # insert code here
        pass
    return labels, centroids

def recompute_centroids(X, k, labels):
    centroids = np.zeros((k, X.shape[1]))
    # insert code here

    return centroids

def recompute_assignments(X, k, centroids):
    labels = np.zeros(X.shape[0])
    # insert code here

    return labels

# generate test set of two gaussian clusters
np.random.seed(42)
X1 = np.random.multivariate_normal([0, 0], [[1, 0], [0, 1]], 100)
X2 = np.random.multivariate_normal([2, 2], [[1, 0], [0, 1]], 100)
X = np.vstack((X1, X2))

# run k-means
k = 2
# take random centroids from data
centroids = X[np.random.choice(X.shape[0], k, replace=False), :]
labels, centroids = k_means(X, k, centroids)

# plot results here

## K-Medians
Now implement K-medians. Here, the centroids are made up of the median of each dimension instead of the average.
Compute the respective within cluster variations for k-means and k-medians and compare them.
Does a better objective also result in a subjectively better clustering?

In [ ]:
def k_medians(X, k, medians,n_iter=100):
    labels = np.zeros(X.shape[0])
    for i in range(n_iter):
        # insert code here
        pass
    return labels, medians

def recompute_medians(X, k, labels):
    medians = np.zeros((k, X.shape[1]))
    # insert code here
    return medians


# generate test set of two gaussian clusters add noise on 
np.random.seed(42)
X1 = np.random.multivariate_normal([0, 0], [[1, 0], [0, 1]], 30)
X2 = np.random.multivariate_normal([2, 2], [[1, 0], [0, 1]], 30)
noise = np.array([[8,8],[8,7],[7,8],[7,7]])
X = np.vstack((X1, X2, noise))

# run k-means
k = 2
centroids = X[np.random.choice(X.shape[0], k, replace=False), :]
labels, centroids = k_means(X, k, centroids)
in_cluster_variation = 0
# compute within cluster variation

# plot results
plt.figure()
plt.title('K-means, W:'+str(in_cluster_variation))
plt.scatter(X[:, 0], X[:, 1], c=labels, s=50)
plt.scatter(centroids[:, 0], centroids[:, 1], c='r', s=100)

# run k-medoids
k = 2
medians = X[np.random.choice(X.shape[0], k, replace=False), :]
labels, medians = k_medians(X, k, medians)
in_cluster_variation = 0
# compute within cluster variation

# plot results here



## Moons
You now get the moons dataset for task 2.1. Perform k-means clustering and analyze the results.

In [ ]:
moons = pd.read_csv('moons.csv').to_numpy()
plt.scatter(moons[:, 0], moons[:, 1], s=50)


## Hierarchical
Instead, we turn to hierarchical clustering.
Implement both single and complete linkage clustering, plot the results and argue which is better.

In [ ]:
from sklearn.cluster import AgglomerativeClustering


## Task 2
You are provided a dataset of 8x8 gray value images.
Perform both PCA and t-SNE embeddings (perplexity=10) to embed the dataset into the two dimensional space and compare the resulting low-dimensional data.
Explain, why each respective method performs better/worse.

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.datasets import load_digits

digits = load_digits()
X = digits.data
# plot sample images
plt.figure(figsize=(10, 1))
for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.imshow(X[i].reshape(8, 8), cmap='gray')
    plt.axis('off')


In [ ]:
# do PCA here

In [ ]:
# do TSNE here

**Analyze the results here**